In [1]:
import os
import shutil
import torch
from batchgenerators.utilities.file_and_folder_operations import load_json, join
from nnunetv2.inference.predict_from_raw_data import nnUNetPredictor
from nnunetv2.evaluation.evaluate_predictions import compute_metrics_on_folder2



In [2]:
torch.cuda.set_device(2)
print(torch.cuda.current_device())

2


In [3]:
# Given 2 json files, this function will compare them and print out the differences
def compare_json_files(json_file1, json_file2):
    json1 = load_json(json_file1)
    json2 = load_json(json_file2)

    # Compare the two JSON files
    differences = {}
    for key in json1.keys():
        if key not in json2:
            differences[key] = (json1[key], None)
        elif json1[key] != json2[key]:
            differences[key] = (json1[key], json2[key])

    for key in json2.keys():
        if key not in json1:
            differences[key] = (None, json2[key])

    return differences
# Example usage
# json_file1 = 'results/250401_21h08m_1472393_BraTS/dataset.json'
# json_file2 = 'data/nnUNet_preprocessed/Dataset111_BraTS19/dataset.json'

# json_file1 = 'results/250401_21h08m_1472393_BraTS/plans.json'
# json_file2 = 'data/nnUNet_preprocessed/Dataset111_BraTS19/nnUNetPlans.json'

# differences = compare_json_files(json_file1, json_file2)
# print("Differences between the two JSON files:\n", differences)

In [4]:
# Experiment configurations
TRAINED_MODELS = {
    'running': {
        'folder': 'results/nnUNet_results/running',
        'cv_fold': '4',
        'preds_name': 'preds_running',
    },
    'UCSF_396': {
        'folder': 'results/250402_17h33m_1488555_UCSF_PGDM',
        'cv_fold': '4',
        'preds_name': 'preds_UCSF_396',
    },
    'BraTS19_267': {
        'folder': 'results/250401_21h08m_1472393_BraTS',
        'cv_fold': 'all',
        'preds_name': 'preds_BraTS19_267',
    },
    'Combined301': {
        'folder': 'results/250408_18h35m_1532958_Combined301',
        'cv_fold': '4',
        'preds_name': 'preds_Combined301',
    },
}

EVALUATION_DATASETS = {
    'LUMIERE_72': {
        'folder': 'data/nnUNet_raw/EvalDataset_LUMIERE',
        'images_dir': 'images',
        'labels_dir': 'labels',
    },
    'CCF_66': {
        'folder': 'data/nnUNet_raw/EvalDataset_CCF',
        'images_dir': 'images',
        'labels_dir': 'labels',
    },
    'BraTS19_68': {
        'folder': 'data/nnUNet_raw/Dataset111_BraTS19',
        'images_dir': 'imagesTs',
        'labels_dir': 'labelsTs',
    },
    '310_Exp1_76': {
        'folder': 'data/nnUNet_raw/Dataset310_Exp1',
        'images_dir': 'imagesTs',
        'labels_dir': 'labelsTs',
    }
}

# Experiment parameters
trained_model_id = 'running'
eval_dataset_id = 'CCF_66'


# Run for each experiment
trained_model_folder = TRAINED_MODELS[trained_model_id]['folder']
trained_model_cv_fold = TRAINED_MODELS[trained_model_id]['cv_fold']
eval_dataset_folder = EVALUATION_DATASETS[eval_dataset_id]['folder']
eval_images_dir = EVALUATION_DATASETS[eval_dataset_id]['images_dir']
eval_labels_dir = EVALUATION_DATASETS[eval_dataset_id]['labels_dir']
preds_name = TRAINED_MODELS[trained_model_id]['preds_name']
seg_preds_dir = join(eval_dataset_folder, preds_name)

predictor = nnUNetPredictor(
    tile_step_size=0.5,
    use_gaussian=True,
    use_mirroring=True,
    perform_everything_on_device=True,
    device=torch.device('cuda', torch.cuda.current_device()),
    verbose=False,
    verbose_preprocessing=False,
    allow_tqdm=True
    )
predictor.initialize_from_trained_model_folder(
    join(trained_model_folder),
    use_folds=trained_model_cv_fold, #(0, ),
    checkpoint_name='checkpoint_best.pth',
)

predictor.predict_from_files(
    join(eval_dataset_folder, eval_images_dir),
    seg_preds_dir,
    save_probabilities=False, overwrite=True,
    num_processes_preprocessing=2, num_processes_segmentation_export=2,
    folder_with_segs_from_prev_stage=None, num_parts=1, part_id=0)


compute_metrics_on_folder2(
    join(eval_dataset_folder, eval_labels_dir),
    seg_preds_dir,
    join(trained_model_folder, 'dataset.json'),
    join(trained_model_folder, 'plans.json'),
    join(eval_dataset_folder, preds_name+'_summary.json'),
)
# clear seg_preds_dir
# shutil.rmtree(seg_preds_dir)


/home/wisc/vichare2/Projects/uumamba/code/uumamba/nnunetv2/inference/predict_from_raw_data.py:87: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(join(

There are 66 cases in the source folder
I am process 0 out of 1 (max process ID is 0, we start counting with 0!)
There are 66 cases that I would like to predict

Predicting 01:
perform_everything_on_device: True


100%|██████████| 8/8 [00:07<00:00,  1.02it/s]


sending off prediction to background worker for resampling and export
done with 01

Predicting 02:
perform_everything_on_device: True


100%|██████████| 8/8 [00:07<00:00,  1.09it/s]


...

sending off prediction to background worker for resampling and export
done with 78

Predicting 79:
perform_everything_on_device: True


100%|██████████| 8/8 [00:07<00:00,  1.09it/s]


sending off prediction to background worker for resampling and export
done with 79

Predicting 81:
perform_everything_on_device: True


100%|██████████| 8/8 [00:07<00:00,  1.09it/s]


sending off prediction to background worker for resampling and export
done with 81
Using <class 'nnunetv2.imageio.simpleitk_reader_writer.SimpleITKIO'> as reader/writer


In [ ]:
# TODO: Visualize the segmentation results
# import nibabel as nib
# import matplotlib.pyplot as plt
# import numpy as np

# # Load image and segmentation
# img = nib.load("image.nii.gz").get_fdata()
# seg = nib.load("segmentation.nii.gz").get_fdata()

# # Pick a slice index (e.g., axial slice)
# slice_idx = 60
# plt.imshow(img[:, :, slice_idx], cmap="gray")
# plt.imshow(seg[:, :, slice_idx], cmap="Reds", alpha=0.5)  # Customize color/opacity
# plt.axis("off")
# plt.show()
